In [1]:
import pandas as pd
df = pd.read_csv('./data/combined_transactions_categorized.csv')

In [3]:
# Ensure Date is datetime and sorted chronologically
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

# Basic calculations
total_income   = df[df['Expense/Income'] == 'Income']['Amount'].sum()
total_expense  = df[df['Expense/Income'] == 'Expense']['Amount'].sum()
net_savings    = total_income - total_expense

# Monthly aggregation
df['Month'] = df['Date'].dt.to_period('M')
monthly = (df.groupby(['Month', 'Expense/Income'])['Amount']
           .sum()
           .unstack(fill_value=0)
           .eval('Net = Income - Expense')
           .round(2))

# Category spending (only expenses)
category_spending = (df[df['Expense/Income'] == 'Expense']
                     .groupby('Category')['Amount']
                     .sum()
                     .sort_values(ascending=False))

# Income sources
income_sources = (df[df['Expense/Income'] == 'Income']
                  .groupby('Name / Description')['Amount']
                  .sum()
                  .sort_values(ascending=False))

# Generate the rich report
report = f"""
FINANCIAL ANALYSIS REPORT (RAG-OPTIMIZED)

Report generated on: {pd.Timestamp('today').strftime('%Y-%m-%d %H:%M')}

1. SUMMARY
• Total transactions: {len(df):,}
• Period covered: {df['Date'].min().date()} to {df['Date'].max().date()} ({(df['Date'].max() - df['Date'].min()).days + 1} days)
• Total Income:  €{total_income:,.2f}
• Total Expenses: €{total_expense:,.2f}
• Net Savings:   €{net_savings:,.2f} {'(Positive)' if net_savings > 0 else '(Deficit)'}

2. MONTHLY BREAKDOWN (Income | Expense | Net)
{monthly.to_string()}

3. AVERAGE MONTHLY FIGURES
• Avg. Monthly Income:  €{monthly['Income'].mean():,.2f}
• Avg. Monthly Expense: €{monthly['Expense'].mean():,.2f}
• Avg. Monthly Net:     €{monthly['Net'].mean():,.2f}

4. TOP EXPENSE CATEGORIES
{category_spending.to_string()}

5. TOP INCOME SOURCES
{income_sources.to_string()}

6. BIGGEST SINGLE TRANSACTIONS (Top 20)
{df.nlargest(20, 'Amount')[['Date', 'Name / Description', 'Amount', 'Expense/Income', 'Category']]
   .to_string(index=False)}

7. KEY OBSERVATIONS & INSIGHTS
• Highest spending category: {category_spending.index[0]} (€{category_spending.iloc[0]:,.2f})
• Largest single expense: {df.loc[df['Amount'].idxmax(), 'Name / Description']} on {df['Date'].max().date()} (€{df['Amount'].max():,.2f})
• Most consistent income: Salary appears {'Salary' in df['Name / Description'].values} times
• Savings trend: {'Improving' if monthly['Net'].is_monotonic_increasing else 'Declining' if monthly['Net'].is_monotonic_decreasing else 'Mixed'}
• Potential data issues: 
  - Income entries with expense categories (e.g., Mutual Fund labeled as Shopping)
  - Missing or inconsistent category names

8. COMPLETE TRANSACTION LIST (for precise lookup)
{df[['Date', 'Name / Description', 'Amount', 'Expense/Income', 'Category']]
   .to_string(index=False)}
"""

# Save the masterpiece
with open('finance_kb.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("RAG-optimized finance report saved as: finance_kb.txt")
print(f"Net savings over the period: €{net_savings:,.2f}")

RAG-optimized finance report saved as: finance_kb.txt
Net savings over the period: €124,366.00
